In [2]:
from google.colab import drive
drive.mount("/content/gdrive")
%cd "/content/gdrive/My Drive/1.0 AI Final Project/LSTM with Framework"
!ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/1.0 AI Final Project/LSTM with Framework
'Data Pre-Processing.ipynb'   dataset_normal.csv   x_train.csv
 dataset_attack.csv	     'LSTM M1.ipynb'


# Dataset


1.   Load the dataset
2.   Preprocess the dataset
   1. Drop unwanted columns and rows
   2. Scaled the data
3.   Split the data into training and test datasets
4.   Save the data into a csv file
  






In [23]:
import pandas as pd

number_of_samples = 50000
columns = ['frame.encap_type', 'frame.len', 'frame.protocols', 'ip.hdr_len',
       'ip.len', 'ip.flags.rb', 'ip.flags.df', 'p.flags.mf', 'ip.frag_offset',
       'ip.ttl', 'ip.proto', 'ip.src', 'ip.dst', 'tcp.srcport', 'tcp.dstport',
       'tcp.len', 'tcp.ack', 'tcp.flags.res', 'tcp.flags.ns', 'tcp.flags.cwr',
       'tcp.flags.ecn', 'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push',
       'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size',
       'tcp.time_delta','class']
df_normal = pd.read_csv('dataset_normal.csv', names = columns, nrows = number_of_samples)
df_attack = pd.read_csv('dataset_attack.csv', names = columns, nrows = number_of_samples)

# Drop the first row
df_normal = df_normal.iloc[1:]
df_attack = df_attack.iloc[1:]

# Drop the first columns
df_normal = df_normal.iloc[: , 1:]
df_attack = df_attack.iloc[: , 1:]

print("Normal Instances Dataset:\n")
print(df_normal.head())
print("Attack Instances Dataset:\n")
print(df_attack.head())

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Normal Instances Dataset:

  frame.len           frame.protocols  ... tcp.time_delta   class
1       206  eth:ethertype:ip:tcp:ssh  ...    0.000000000  normal
2        60      eth:ethertype:ip:tcp  ...    0.000537000  normal
3        60      eth:ethertype:ip:tcp  ...    0.000155000  normal
4       774  eth:ethertype:ip:tcp:ssh  ...    0.004483000  normal
5       774      eth:ethertype:ip:tcp  ...    0.001321000  normal

[5 rows x 29 columns]
Attack Instances Dataset:

  frame.len            frame.protocols  ... tcp.time_delta   class
1       212  eth:ethertype:ip:udp:data  ...              0  attack
2       212  eth:ethertype:ip:udp:data  ...              0  attack
3       212  eth:ethertype:ip:udp:data  ...              0  attack
4       212  eth:ethertype:ip:udp:data  ...              0  attack
5        62  eth:ethertype:ip:udp:data  ...              0  attack

[5 rows x 29 columns]


# Drop unwanted colums


In [24]:
df_normal = df_normal.drop(['ip.src', 'ip.dst','frame.protocols'],axis=1)
df_attack = df_attack.drop(['ip.src', 'ip.dst','frame.protocols'],axis=1)

print("Normal Instances Dataset:\n")
print(df_normal.head())
print("Attack Instances Dataset:\n")
print(df_attack.head())

Normal Instances Dataset:

  frame.len ip.hdr_len ip.len  ... tcp.window_size tcp.time_delta   class
1       206         20    192  ...           16697    0.000000000  normal
2        60         20     40  ...            7504    0.000537000  normal
3        60         20     40  ...            7504    0.000155000  normal
4       774         20    760  ...            7504    0.004483000  normal
5       774         20    760  ...            7504    0.001321000  normal

[5 rows x 26 columns]
Attack Instances Dataset:

  frame.len ip.hdr_len ip.len  ... tcp.window_size tcp.time_delta   class
1       212         20    198  ...               0              0  attack
2       212         20    198  ...               0              0  attack
3       212         20    198  ...               0              0  attack
4       212         20    198  ...               0              0  attack
5        62         20     48  ...               0              0  attack

[5 rows x 26 columns]


# Spilit the features from the prediction

In [25]:
import numpy as np

features=[ 'frame.len', 'ip.hdr_len',
       'ip.len', 'ip.flags.rb', 'ip.flags.df', 'p.flags.mf', 'ip.frag_offset',
       'ip.ttl', 'ip.proto', 'tcp.srcport', 'tcp.dstport',
       'tcp.len', 'tcp.ack', 'tcp.flags.res', 'tcp.flags.ns', 'tcp.flags.cwr',
       'tcp.flags.ecn', 'tcp.flags.urg', 'tcp.flags.ack', 'tcp.flags.push',
       'tcp.flags.reset', 'tcp.flags.syn', 'tcp.flags.fin', 'tcp.window_size',
       'tcp.time_delta']

# Get the feature columns (features)
x_normal = df_normal[features].values
x_attack = df_attack[features].values

# Get the prediction column
y_normal = df_normal['class']
y_attack = df_attack['class']

# Concaenate the normal and attack traffic
features = np.concatenate((x_normal,x_attack))
predictions = np.concatenate((y_normal,y_attack))

print("Features:\n")
print(features)
print("Predictions:\n")
print(predictions)

Features:

[['206' '20' '192' ... '0' '16697' '0.000000000']
 ['60' '20' '40' ... '0' '7504' '0.000537000']
 ['60' '20' '40' ... '0' '7504' '0.000155000']
 ...
 [1414 20 1400 ... 0 64030 0.217445]
 [154 20 140 ... 0 64030 6e-06]
 [60 20 40 ... 0 9140 0.001324]]
Predictions:

['normal' 'normal' 'normal' ... 'attack' 'attack' 'attack']


# Standarize the data

In [26]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
scaler.fit(features)
features = scaler.transform(features)

print("Scaled Data:\n")
print(features)

Scaled Data:

[[-6.35532920e-01  3.42256331e-02 -6.31863337e-01 ... -1.29726073e-01
   2.07215413e-01 -6.79980187e-02]
 [-8.59639002e-01  3.42256331e-02 -8.64702227e-01 ... -1.29726073e-01
  -2.98854094e-01 -6.78328062e-02]
 [-8.59639002e-01  3.42256331e-02 -8.64702227e-01 ... -1.29726073e-01
  -2.98854094e-01 -6.79503317e-02]
 ...
 [ 1.21871466e+00  3.42256331e-02  1.21859311e+00 ... -1.29726073e-01
   2.81287059e+00 -1.09924641e-03]
 [-7.15351525e-01  3.42256331e-02 -7.11518747e-01 ... -1.29726073e-01
   2.81287059e+00 -6.79961727e-02]
 [-8.59639002e-01  3.42256331e-02 -8.64702227e-01 ... -1.29726073e-01
  -2.08793209e-01 -6.75906790e-02]]


# Change class label from categorical to numerical data

In [27]:
print("Predictions: \n")
print(predictions)

for i in range(0,len(predictions)):
  if predictions[i] == 'attack':
    predictions[i]=0
  else:
    predictions[i]=1

print("Predictions: \n")
print(predictions)

Predictions: 

['normal' 'normal' 'normal' ... 'attack' 'attack' 'attack']
Predictions: 

[1 1 1 ... 0 0 0]


# Change instances from conventional packet-based to window-based 

In [32]:
feat = len(features[0])
samples = features.shape[0]
train_len = 25
input_len = samples - train_len
I = np.zeros((samples - train_len, train_len, feat))

for i in range(input_len):
    temp = np.zeros((train_len, feat))
    for j in range(i, i + train_len - 1):
        temp[j-i] = features[j]
    I[i] = temp

# Split the data into training and test datasets

In [29]:
features.shape

(99998, 25)

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(I, predictions[25:99998], test_size = 0.2)

# Save into binary files

In [35]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

print("Start Saving:\n")
np.save("x_train.npy", X_train)
np.save("x_test.npy", X_test)

np.save("y_train.npy", Y_train)
np.save("y_test.npy", Y_test)
print("End Saving:\n")

(79978, 25, 25)
(19995, 25, 25)
(79978,)
(19995,)
Start Saving:

End Saving:

